# Notebook 03 – Backend con FastAPI siguiendo arquitectura MVC

En este notebook se construye el backend de la aplicación utilizando FastAPI,
siguiendo el patrón arquitectónico Modelo–Vista–Controlador (MVC).

El objetivo es exponer la generación de contenido mediante una API REST
que posteriormente será consumida por un frontend en React.


## Arquitectura MVC en este proyecto

- Model: Esquemas de datos (Pydantic)
- Controller: Endpoints FastAPI
- Service: Lógica de negocio (LLMs, prompts, RAG)
- View: Frontend React (fuera del backend)

Esta separación permite un código más mantenible, escalable y testeable.


📁 backend/app/schema/content_request.py

In [1]:
from pydantic import BaseModel

class ContentRequest(BaseModel):
    tema: str
    plataforma: str
    audiencia: str
    tono: str


📁 backend/app/schema/content_response.py

In [2]:
from pydantic import BaseModel

class ContentResponse(BaseModel):
    content: str


## Capa de Servicios (Service Layer)

La capa de servicios contiene la lógica de negocio.
Aquí se comunica con el modelo de lenguaje (LLM).


📁 backend/app/services/content_service.py

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

class ContentService:

    def __init__(self):
        self.llm = ChatGroq(
            model="llama3-8b-8192",
            api_key=os.getenv("GROQ_API_KEY")
        )

    def generar_contenido(
        self,
        tema: str,
        plataforma: str,
        audiencia: str,
        tono: str
    ) -> str:

        prompt = f"""
        Eres un experto creador de contenido digital.

        Tema: {tema}
        Plataforma: {plataforma}
        Audiencia: {audiencia}
        Tono: {tono}

        Genera un texto listo para publicar, adaptado a la plataforma indicada.
        """

        response = self.llm.invoke(prompt)
        return response.content


## Controlador (Controller)

El controlador expone la lógica del servicio a través de una API REST.


📁 backend/app/controllers/content_controller.py

In [ ]:
from fastapi import APIRouter
from app.schema.content_request import ContentRequest
from app.schema.content_response import ContentResponse
from app.services.content_service import ContentService

router = APIRouter()
service = ContentService()

@router.post("/generate", response_model=ContentResponse)
def generate_content(request: ContentRequest):
    content = service.generar_contenido(
        tema=request.tema,
        plataforma=request.plataforma,
        audiencia=request.audiencia,
        tono=request.tono
    )
    return ContentResponse(content=content)


📁 backend/app/main.py

In [ ]:
from fastapi import FastAPI
from app.controllers.content_controller import router as content_router

app = FastAPI(title="Generador de Contenido con LLMs")

app.include_router(content_router, prefix="/api")
